# Summary of Pandas commands and the equivalent SQL expression

##### Groupby

- **Basic aggregating:**

   <pre><code>df.groupby('col1',sort=False).agg({'col2':function}).reset_index()</code></pre> 

where the function could be sum, mean, min, max, lambda x: x.nunique(dropna=False), lambda x: x.shape[0]. The reset_index command forces a new index and so that we have no multi-indexing problems. If we do multiple aggregations on the same column then we will also need to reset the columns.

   <pre><code> select sum(a.col2), avg(a.col2), min(a.col2), max(a.col2) from df a groupby a.col1 </code></pre>
   
Note that when doing a count of a column it will exclude nulls, when doing count of all it will count nulls as well. This is important when thinking about counting the number of rows in a group since you may have to do a group by first to get unique rows and then do count(*)

Note that you can group on more than one column

- **Group Filter:**

    <pre><code> df.groupby('col1',sort=False).filter(lambda x: x['col1'].sum()<2)  </code></pre> 
    
Filter has access to the entire data frame for the group. It returns the original data frame with those groups that don't meet the criteria removed.
    
  <pre><code> select a.* from data a inner join (select b.prod1 from data b group by b.prod1 having sum(b.month)>6) c on a.prod1=c.prod1 </code></pre> 
  
The filter version for sql is the having clause. This only returns the group not the entire data frame so might have to do an inner join
 
- **Group Transform:**

<pre><code> df.groupby('col1',sort=False).transform(lambda x: x - x.mean()) </code></pre>  

Operates on each column separately so if you want to look at one column you need to specify that column before the transform

<pre><code> select prod1, month - avg(month) over (partition by prod1) from data </code></pre> 

- **General operations on a group:**

<pre><code>data.groupby('prod1').apply(lambda x: x['pop1'] - x['month'].sum()).reset_index()<code></pre> 

The apply function will have access to a data frame by group. If you return a single column might be best to get the values directly. You can also return an entire data frame however.

<pre><code> select prod1, pop1 - (sum(month) over (partition by prod1)) from data </code></pre>


- **Row Number by group**

<pre><code> data['rank'] = data.groupby('prod1').cumcount() + 1 <code></pre>

<pre><code> select prod1, pop1, month, row_number() over (partition by prod1) as rank from data <code></pre>

Rank is similar for both of these

Using this you can order by and get the second row

- **Percentile and Binning by group**
<pre><code> select prod1, month, percent_rank() over (partition by prod1 order by month) ,ntile(2) over (partition by prod1 order by month) from data<code></pre>

- **Rolling window**

<pre><code> data.groupby('prod1',sort=False).month.rolling(3).sum().values<code></pre>

Instead of sum here you can do apply and have access to anything in that window

<pre><code> select prod1, pop1, month, sum(month) over (partition by prod1 rows 2 preceding) from data<code></pre>
<pre><code> select prod1, pop1, month, sum(month) over (rows unbounded preceding) from data<code></pre>

- **Calculate rolling difference**

<pre><code> data.groupby('prod1',sort=False).month.rolling(2).apply(lambda x: x[1] - x[0]).values <code></pre>
    
    select prod1, pop1, month, month - lag(month,1) over (partition by prod1) from data




##### Merge

<pre><code> df1.merge(df2, on='col1', how='inner') <code></pre>

<pre><code> select * from data a inner join data b on a.prod1=b.prod1 <code></pre>

Note that you can do an inequality sign on the join, that way you can get at cumulative sum for example (better ways of doing that though)  

Left join, full join (keeps all rows, might match on some on other they will just fill with null), left semi join not an option in sql but can do left join and filter out non null rows (although this is actually an inner join, really to mimic a left semi join you need to do an exist clause where you are filtering the left table down to the rows that are also in the right table).

When you do a select * before doing the join it will select all columns from both tables

##### Concat


<pre><code> pd.concat([df1,df2],ignore_index=True)<code></pre>
    
<pre><code> select * from data a union all select * from data b <code></pre>

Just "union" will only return distinct rows

##### Misc.

- **Sort**:
<pre><code> df.sort_values(['col1','col2'], ascending=False) <code></pre>

<pre><code> select * from data order by prod1, month <code></pre>

- **SQL Exists**
<pre><code> select prod1, month from data b where exists (select a.month from (select month from data c where c.month>2) a where a.month=b.month) <code></pre>

select * from log_1 b where exists (select distinct(user_id) from log_1 where date>340) c where b.user_id=c.user_id

The second where clause connects the first where clause. So we say for each row, does the subquery return one or more records and that is connected through that second where clause.

- **SQL Case**

<pre><code> select prod1, month, case when month==1 then "Jan" when month==2 then "Feb" else "other" end as month_name from data <code></pre>

- **NA**

- df.fill_na(0) : fill all na values with 0
- df.isna() : Return Boolean dataframe where true is for na

<pre><code> select prod1, month is null from data <code></pre>
<pre><code> select pop1, prod1, coalesce(month, 0) from data <code></pre>

- **Other**
- df.explode('col1') : Explode column 1
- df.groupby('col2').agg({'col1':lambda x: x.values})
- df.drop_duplicates() : return unique rows
- df['col1'].unique() : return unique values

- subtract 1 from column:
<pre><code> select pop1, month - 1 from data<code></pre>


Mistakes: 
Not grouping over everything that needs to be grouped over
reset_index when I've got a name collision


# Demo
Throughout these exercies first approach starts with a pandas operation and the second approach is SQL

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import numpy as np

#### Get sql engine going

In [2]:
engine = create_engine('sqlite://', echo=False)
inspector = inspect(engine)

### 0. Initial Data

Create example data

In [3]:
data = pd.DataFrame()
data['prod1'] = ['A','A','A','A','B','B','B','B','B','B','C','C','C']
data['pop1'] = [1,2,1,3,1,2,3,4,3,4,1,2,3]
data['month'] = [1,2,4,3,1,2,3,4,5,6,1,3,np.nan]

In [4]:
data

,prod1,pop1,month
0,A,1,1.0
1,A,2,2.0
2,A,1,4.0
3,A,3,3.0
4,B,1,1.0
5,B,2,2.0
6,B,3,3.0
7,B,4,4.0
8,B,3,5.0
9,B,4,6.0


Writes data to sql data base

In [5]:
data.to_sql('data', con=engine)

In [6]:
engine.execute("SELECT * from data").fetchall()

[(0, 'A', 1, 1.0),
 (1, 'A', 2, 2.0),
 (2, 'A', 1, 4.0),
 (3, 'A', 3, 3.0),
 (4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0),
 (6, 'B', 3, 3.0),
 (7, 'B', 4, 4.0),
 (8, 'B', 3, 5.0),
 (9, 'B', 4, 6.0),
 (10, 'C', 1, 1.0),
 (11, 'C', 2, 3.0),
 (12, 'C', 3, None)]

In [17]:
engine.execute('select prod1, sum(month) over(rows unbounded preceding),month from data').fetchall()

[('A', 1.0, 1.0),
 ('A', 3.0, 2.0),
 ('A', 7.0, 4.0),
 ('A', 10.0, 3.0),
 ('B', 11.0, 1.0),
 ('B', 13.0, 2.0),
 ('B', 16.0, 3.0),
 ('B', 20.0, 4.0),
 ('B', 25.0, 5.0),
 ('B', 31.0, 6.0),
 ('C', 32.0, 1.0),
 ('C', 35.0, 3.0),
 ('C', 35.0, None)]

In [92]:
engine.execute("select prod1, month, ntile(2) over (partition by prod1 order by month) from data ").fetchall()

[('A', 1.0, 1),
 ('A', 2.0, 1),
 ('A', 3.0, 2),
 ('A', 4.0, 2),
 ('B', 1.0, 1),
 ('B', 2.0, 1),
 ('B', 3.0, 1),
 ('B', 4.0, 2),
 ('B', 5.0, 2),
 ('B', 6.0, 2),
 ('C', None, 1),
 ('C', 1.0, 1),
 ('C', 3.0, 2)]

In [16]:
engine.execute('select prod1, month, lag(month,1) over () from data').fetchall()

[('A', 1.0, None),
 ('A', 2.0, 1.0),
 ('A', 4.0, 2.0),
 ('A', 3.0, 4.0),
 ('B', 1.0, 3.0),
 ('B', 2.0, 1.0),
 ('B', 3.0, 2.0),
 ('B', 4.0, 3.0),
 ('B', 5.0, 4.0),
 ('B', 6.0, 5.0),
 ('C', 1.0, 6.0),
 ('C', 3.0, 1.0),
 ('C', None, 3.0)]

### 1. Groupby practice

#### How many nonnull months are there for each product?
Count will count all the nonnull rows of month. When we do agg we are doing the function over the column we selected. Also note that when we do reset_index it will put the grouped column as its own column instead of the index

In [7]:
data.groupby('prod1').agg({'month':'count'}).reset_index()

,prod1,month
0,A,4
1,B,6
2,C,2


In [8]:
engine.execute('select prod1, count(month) from data group by prod1').fetchall()

[('A', 4), ('B', 6), ('C', 2)]

#### How many total rows are there for each product?
If you want to count all rows (even with nulls) you need to do x.shape

In [9]:
data.groupby('prod1').agg({'pop1':lambda x: x.shape[0]})

,pop1
prod1,
A,4
B,6
C,3


Doing a count(*) counts the number of rows whereas counting a column counts on non-null

In [10]:
engine.execute('select prod1, count(*) from data group by prod1').fetchall()

[('A', 4), ('B', 6), ('C', 3)]

#### How many unique popularity scores are there for each product?
Use nunique - if you want to count the distinct nulls as well pass the appropriate argument

In [16]:
data.groupby('prod1').agg({'pop1':lambda x: x.nunique(dropna=False)})

,pop1
prod1,
A,3
B,4
C,3


Have to deal with the issue that count(*) is the only way to count nulls

In [17]:
engine.execute('select a.prod1, count(*) from (select prod1, pop1 from data group by prod1, month) a group by a.prod1').fetchall()

[('A', 4), ('B', 6), ('C', 3)]

#### For each month what is the average popularity?

In [30]:
data.groupby('month').agg({'pop1':'mean'})

,pop1
month,
1.0,1.000000
2.0,2.000000
3.0,2.666667
4.0,2.500000
5.0,3.000000
6.0,4.000000


In [28]:
engine.execute('select month, avg(pop1) from data group by month').fetchall()

[(None, 3.0),
 (1.0, 1.0),
 (2.0, 2.0),
 (3.0, 2.6666666666666665),
 (4.0, 2.5),
 (5.0, 3.0),
 (6.0, 4.0)]

#### For each month and product how many unique popularity numbers are there?
Grouping on two columns.

In [11]:
data_group = data.groupby(['month','prod1']).agg({'pop1':['sum',lambda x: x.nunique(dropna=False)]}).reset_index()
data_group.columns = ['month','prod1','pop1_sum','pop1_nunique']

In [12]:
data_group

,month,prod1,pop1_sum,pop1_nunique
0,1.0,A,1,1
1,1.0,B,1,1
2,1.0,C,1,1
3,2.0,A,2,1
4,2.0,B,2,1
5,3.0,A,3,1
6,3.0,B,3,1
7,3.0,C,2,1
8,4.0,A,1,1
9,4.0,B,4,1


In [33]:
engine.execute('select month, prod1, count(distinct(pop1)), sum(pop1) from data group by month, prod1').fetchall()


[(None, 'C', 1, 3),
 (1.0, 'A', 1, 1),
 (1.0, 'B', 1, 1),
 (1.0, 'C', 1, 1),
 (2.0, 'A', 1, 2),
 (2.0, 'B', 1, 2),
 (3.0, 'A', 1, 3),
 (3.0, 'B', 1, 3),
 (3.0, 'C', 1, 2),
 (4.0, 'A', 1, 1),
 (4.0, 'B', 1, 4),
 (5.0, 'B', 1, 3),
 (6.0, 'B', 1, 4)]

#### Filter groups
This filters our dataset down to those groups that meet the criteria. The entire data frame is passed into the function "filter" partitioned by the group

In [31]:
data.groupby('prod1').filter(lambda x: x['month'].sum()>6)

,prod1,pop1,month
0,A,1,1.0
1,A,2,2.0
2,A,1,4.0
3,A,3,3.0
4,B,1,1.0
5,B,2,2.0
6,B,3,3.0
7,B,4,4.0
8,B,3,5.0
9,B,4,6.0


In [59]:
engine.execute("select a.* from data a inner join (select b.prod1 from data b group by b.prod1 having sum(b.month)>6) c on a.prod1=c.prod1 ").fetchall()


[(0, 'A', 1, 1.0),
 (1, 'A', 2, 2.0),
 (2, 'A', 1, 4.0),
 (3, 'A', 3, 3.0),
 (4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0),
 (6, 'B', 3, 3.0),
 (7, 'B', 4, 4.0),
 (8, 'B', 3, 5.0),
 (9, 'B', 4, 6.0)]

#### Transform
Each column of the data frame is passed into the "transform" function.

In [60]:
data['standardized'] = data.groupby('prod1', sort=False).month.transform(lambda x: x - x.mean())

In [61]:
data

,prod1,pop1,month,standardized
0,A,1,1.0,-1.5
1,A,2,2.0,-0.5
2,A,1,4.0,1.5
3,A,3,3.0,0.5
4,B,1,1.0,-2.5
5,B,2,2.0,-1.5
6,B,3,3.0,-0.5
7,B,4,4.0,0.5
8,B,3,5.0,1.5
9,B,4,6.0,2.5


In [66]:
engine.execute('select prod1, month - avg(month) over (partition by prod1) from data').fetchall()

[('A', -1.5),
 ('A', -0.5),
 ('A', 1.5),
 ('A', 0.5),
 ('B', -2.5),
 ('B', -1.5),
 ('B', -0.5),
 ('B', 0.5),
 ('B', 1.5),
 ('B', 2.5),
 ('C', -1.0),
 ('C', 1.0),
 ('C', None)]

Start Here --> #### Apply
The entire dataset is passed in partitioned by groups

In [21]:
data.groupby('prod1').apply(lambda x: x['pop1'] - x['month'].sum()).reset_index()

,prod1,level_1,pop1
0,A,0,-9.0
1,A,1,-8.0
2,A,2,-9.0
3,A,3,-7.0
4,B,4,-20.0
5,B,5,-19.0
6,B,6,-18.0
7,B,7,-17.0
8,B,8,-18.0
9,B,9,-17.0


In [25]:
engine.execute('select prod1, pop1 - (sum(month) over (partition by prod1)) from data').fetchall()

[('A', -9.0),
 ('A', -8.0),
 ('A', -9.0),
 ('A', -7.0),
 ('B', -20.0),
 ('B', -19.0),
 ('B', -18.0),
 ('B', -17.0),
 ('B', -18.0),
 ('B', -17.0),
 ('C', -3.0),
 ('C', -2.0),
 ('C', -1.0)]

#### Cumcount
Just add a range of values to each group

In [5]:
data['rank'] = data.groupby('prod1').cumcount() + 1

In [6]:
data

,prod1,pop1,month,rank
0,A,1,1,1
1,A,2,2,2
2,A,1,4,3
3,A,3,3,4
4,B,1,1,1
5,B,2,2,2
6,B,3,3,3
7,B,4,4,4
8,B,3,5,5
9,B,4,6,6


In [68]:
engine.execute('select prod1, pop1, month, row_number() over (partition by prod1) from data').fetchall()

[('A', 1, 1.0, 1),
 ('A', 2, 2.0, 2),
 ('A', 1, 4.0, 3),
 ('A', 3, 3.0, 4),
 ('B', 1, 1.0, 1),
 ('B', 2, 2.0, 2),
 ('B', 3, 3.0, 3),
 ('B', 4, 4.0, 4),
 ('B', 3, 5.0, 5),
 ('B', 4, 6.0, 6),
 ('C', 1, 1.0, 1),
 ('C', 2, 3.0, 2),
 ('C', 3, None, 3)]

#### Rank
Rank according to some column. Ties go to the first one

In [81]:
data['rank1'] = data.groupby('prod1').pop1.rank(method='first')

In [82]:
data

,prod1,pop1,month,standardized,rank,rank1
0,A,1,1,-1.5,1,1.0
1,A,2,2,-0.5,2,3.0
2,A,1,4,1.5,3,2.0
3,A,3,3,0.5,4,4.0
4,B,1,1,-2.5,1,1.0
5,B,2,2,-1.5,2,2.0
6,B,3,3,-0.5,3,3.0
7,B,4,4,0.5,4,5.0
8,B,3,5,1.5,5,4.0
9,B,4,6,2.5,6,6.0


#### Percentile

In [38]:
engine.execute('select prod1, month, percent_rank() over (partition by prod1 order by month) ,ntile(2) over (partition by prod1 order by month) from data').fetchall()

[('A', 1.0, 0.0, 1),
 ('A', 2.0, 0.3333333333333333, 1),
 ('A', 3.0, 0.6666666666666666, 2),
 ('A', 4.0, 1.0, 2),
 ('B', 1.0, 0.0, 1),
 ('B', 2.0, 0.2, 1),
 ('B', 3.0, 0.4, 1),
 ('B', 4.0, 0.6, 2),
 ('B', 5.0, 0.8, 2),
 ('B', 6.0, 1.0, 2),
 ('C', None, 0.0, 1),
 ('C', 1.0, 0.5, 1),
 ('C', 3.0, 1.0, 2)]

#### Rolling window
Like the transformation function it opperates on each column separately. Meaning if it passes us a data frame it will work on each column seperately. We can use apply in conjunction with this function to get different behaviour.

In [97]:
data['rolling_sum'] = data.groupby('prod1',sort=False).month.rolling(3).sum().values

In [98]:
data

,prod1,pop1,month,standardized,rank,rank1,rolling_sum
0,A,1,1,-1.5,1,1.0,NaN
1,A,2,2,-0.5,2,3.0,NaN
2,A,1,4,1.5,3,2.0,7.0
3,A,3,3,0.5,4,4.0,9.0
4,B,1,1,-2.5,1,1.0,NaN
5,B,2,2,-1.5,2,2.0,NaN
6,B,3,3,-0.5,3,3.0,6.0
7,B,4,4,0.5,4,5.0,9.0
8,B,3,5,1.5,5,4.0,12.0
9,B,4,6,2.5,6,6.0,15.0


In [11]:
engine.execute('select prod1, pop1, month, sum(month) over (partition by prod1 rows 2 preceding) from data').fetchall()


[('A', 1, 1.0, 1.0),
 ('A', 2, 2.0, 3.0),
 ('A', 1, 4.0, 7.0),
 ('A', 3, 3.0, 9.0),
 ('B', 1, 1.0, 1.0),
 ('B', 2, 2.0, 3.0),
 ('B', 3, 3.0, 6.0),
 ('B', 4, 4.0, 9.0),
 ('B', 3, 5.0, 12.0),
 ('B', 4, 6.0, 15.0),
 ('C', 1, 1.0, 1.0),
 ('C', 2, 3.0, 4.0),
 ('C', 3, None, 4.0)]

In [30]:
data['diff_roll'] = data.groupby('prod1',sort=False).month.rolling(2).apply(lambda x: x[1] - x[0]).values

/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [32]:
data

,prod1,pop1,month,diff_roll
0,A,1,1.0,NaN
1,A,2,2.0,1.0
2,A,1,4.0,2.0
3,A,3,3.0,-1.0
4,B,1,1.0,NaN
5,B,2,2.0,1.0
6,B,3,3.0,1.0
7,B,4,4.0,1.0
8,B,3,5.0,1.0
9,B,4,6.0,1.0


In [33]:
engine.execute('select prod1, pop1, month, month - lag(month,1) over (partition by prod1) from data').fetchall()

[('A', 1, 1.0, None),
 ('A', 2, 2.0, 1.0),
 ('A', 1, 4.0, 2.0),
 ('A', 3, 3.0, -1.0),
 ('B', 1, 1.0, None),
 ('B', 2, 2.0, 1.0),
 ('B', 3, 3.0, 1.0),
 ('B', 4, 4.0, 1.0),
 ('B', 3, 5.0, 1.0),
 ('B', 4, 6.0, 1.0),
 ('C', 1, 1.0, None),
 ('C', 2, 3.0, 2.0),
 ('C', 3, None, None)]

### 2. Join Practice

In [5]:
data.merge(data, on='product',how='inner')

,product,pop_x,month_x,pop_y,month_y
0,A,1,1,1,1
1,A,1,1,2,2
2,A,1,1,1,4
3,A,1,1,3,3
4,A,2,2,1,1
...,...,...,...,...,...
56,C,2,3,2,3
57,C,2,3,3,2
58,C,3,2,1,1
59,C,3,2,2,3


In [42]:
engine.execute('select * from data a left join data b on a.prod1=b.prod1' ).fetchall()

[(0, 'A', 1, 1.0, 0, 'A', 1, 1.0),
 (0, 'A', 1, 1.0, 1, 'A', 2, 2.0),
 (0, 'A', 1, 1.0, 2, 'A', 1, 4.0),
 (0, 'A', 1, 1.0, 3, 'A', 3, 3.0),
 (1, 'A', 2, 2.0, 0, 'A', 1, 1.0),
 (1, 'A', 2, 2.0, 1, 'A', 2, 2.0),
 (1, 'A', 2, 2.0, 2, 'A', 1, 4.0),
 (1, 'A', 2, 2.0, 3, 'A', 3, 3.0),
 (2, 'A', 1, 4.0, 0, 'A', 1, 1.0),
 (2, 'A', 1, 4.0, 1, 'A', 2, 2.0),
 (2, 'A', 1, 4.0, 2, 'A', 1, 4.0),
 (2, 'A', 1, 4.0, 3, 'A', 3, 3.0),
 (3, 'A', 3, 3.0, 0, 'A', 1, 1.0),
 (3, 'A', 3, 3.0, 1, 'A', 2, 2.0),
 (3, 'A', 3, 3.0, 2, 'A', 1, 4.0),
 (3, 'A', 3, 3.0, 3, 'A', 3, 3.0),
 (4, 'B', 1, 1.0, 4, 'B', 1, 1.0),
 (4, 'B', 1, 1.0, 5, 'B', 2, 2.0),
 (4, 'B', 1, 1.0, 6, 'B', 3, 3.0),
 (4, 'B', 1, 1.0, 7, 'B', 4, 4.0),
 (4, 'B', 1, 1.0, 8, 'B', 3, 5.0),
 (4, 'B', 1, 1.0, 9, 'B', 4, 6.0),
 (5, 'B', 2, 2.0, 4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0, 5, 'B', 2, 2.0),
 (5, 'B', 2, 2.0, 6, 'B', 3, 3.0),
 (5, 'B', 2, 2.0, 7, 'B', 4, 4.0),
 (5, 'B', 2, 2.0, 8, 'B', 3, 5.0),
 (5, 'B', 2, 2.0, 9, 'B', 4, 6.0),
 (6, 'B', 3, 3.0, 4,

In [ ]:
engine.execute('select * from data a inner join data b on a.prod1=b.prod1' ).fetchall()

#### Union

In [6]:
pd.concat([data,data],ignore_index=True)

,product,pop,month
0,A,1,1
1,A,2,2
2,A,1,4
3,A,3,3
4,B,1,1
5,B,2,2
6,B,3,3
7,B,4,4
8,B,3,5
9,B,4,6


In [21]:
engine.execute('select * from data a union all select * from data b').fetchall()

[(0, 'A', 1, 1.0),
 (1, 'A', 2, 2.0),
 (2, 'A', 1, 4.0),
 (3, 'A', 3, 3.0),
 (4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0),
 (6, 'B', 3, 3.0),
 (7, 'B', 4, 4.0),
 (8, 'B', 3, 5.0),
 (9, 'B', 4, 6.0),
 (10, 'C', 1, 1.0),
 (11, 'C', 2, 3.0),
 (12, 'C', 3, None),
 (0, 'A', 1, 1.0),
 (1, 'A', 2, 2.0),
 (2, 'A', 1, 4.0),
 (3, 'A', 3, 3.0),
 (4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0),
 (6, 'B', 3, 3.0),
 (7, 'B', 4, 4.0),
 (8, 'B', 3, 5.0),
 (9, 'B', 4, 6.0),
 (10, 'C', 1, 1.0),
 (11, 'C', 2, 3.0),
 (12, 'C', 3, None)]

In [44]:
engine.execute('select * from data a union select * from data b').fetchall()

[(0, 'A', 1, 1.0),
 (1, 'A', 2, 2.0),
 (2, 'A', 1, 4.0),
 (3, 'A', 3, 3.0),
 (4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0),
 (6, 'B', 3, 3.0),
 (7, 'B', 4, 4.0),
 (8, 'B', 3, 5.0),
 (9, 'B', 4, 6.0),
 (10, 'C', 1, 1.0),
 (11, 'C', 2, 3.0),
 (12, 'C', 3, None)]

### 3. Misc.

#### Filtering

In [24]:
data[(data.month==1)]

,product,pop,month
0,A,1,1
4,B,1,1
10,C,1,1


In [23]:
engine.execute('select * from data where month==1').fetchall()

[(0, 'A', 1, 1.0), (4, 'B', 1, 1.0), (10, 'C', 1, 1.0)]

#### Sorting

In [26]:
data.sort_values(['product','month'])

,product,pop,month
0,A,1,1
1,A,2,2
3,A,3,3
2,A,1,4
4,B,1,1
5,B,2,2
6,B,3,3
7,B,4,4
8,B,3,5
9,B,4,6


In [24]:
engine.execute('select * from data order by prod1, month').fetchall()

[(0, 'A', 1, 1.0),
 (1, 'A', 2, 2.0),
 (3, 'A', 3, 3.0),
 (2, 'A', 1, 4.0),
 (4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0),
 (6, 'B', 3, 3.0),
 (7, 'B', 4, 4.0),
 (8, 'B', 3, 5.0),
 (9, 'B', 4, 6.0),
 (12, 'C', 3, None),
 (10, 'C', 1, 1.0),
 (11, 'C', 2, 3.0)]

#### SQL Exists

In [55]:
engine.execute("select prod1, month from data b where exists (select a.month from (select month from data c where c.month>2) a where a.month=b.month) ").fetchall()


[('A', 4.0),
 ('A', 3.0),
 ('B', 3.0),
 ('B', 4.0),
 ('B', 5.0),
 ('B', 6.0),
 ('C', 3.0)]

#### SQL Case

In [60]:
engine.execute('select prod1, month, case when month==1 then "Jan" when month==2 then "Feb" else "other" end as month_name from data').fetchall()

[('A', 1.0, 'Jan'),
 ('A', 2.0, 'Feb'),
 ('A', 4.0, 'other'),
 ('A', 3.0, 'other'),
 ('B', 1.0, 'Jan'),
 ('B', 2.0, 'Feb'),
 ('B', 3.0, 'other'),
 ('B', 4.0, 'other'),
 ('B', 5.0, 'other'),
 ('B', 6.0, 'other'),
 ('C', 1.0, 'Jan'),
 ('C', 3.0, 'other'),
 ('C', None, 'other')]

#### SQL Like

In [66]:
engine.execute('select * from data where prod1 like "%A"').fetchall()

[(0, 'A', 1, 1.0), (1, 'A', 2, 2.0), (2, 'A', 1, 4.0), (3, 'A', 3, 3.0)]

#### Return unique values

In [30]:
data.month.unique()

array([1, 2, 4, 3, 5, 6])

In [25]:
engine.execute('select distinct(month) from data').fetchall()

[(1.0,), (2.0,), (4.0,), (3.0,), (5.0,), (6.0,), (None,)]

In [31]:
data.drop_duplicates()

,product,pop,month
0,A,1,1
1,A,2,2
2,A,1,4
3,A,3,3
4,B,1,1
5,B,2,2
6,B,3,3
7,B,4,4
8,B,3,5
9,B,4,6


In [26]:
engine.execute('select * from data group by prod1, pop1, month').fetchall()

[(0, 'A', 1, 1.0),
 (2, 'A', 1, 4.0),
 (1, 'A', 2, 2.0),
 (3, 'A', 3, 3.0),
 (4, 'B', 1, 1.0),
 (5, 'B', 2, 2.0),
 (6, 'B', 3, 3.0),
 (8, 'B', 3, 5.0),
 (7, 'B', 4, 4.0),
 (9, 'B', 4, 6.0),
 (10, 'C', 1, 1.0),
 (11, 'C', 2, 3.0),
 (12, 'C', 3, None)]

#### Na's

In [61]:
data.isna()

,prod1,pop1,month,diff_roll
0,False,False,False,True
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,True
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [62]:
engine.execute('select * from data where month is null').fetchall()

[(12, 'C', 3, None)]

In [63]:
engine.execute('select prod1, month is null from data').fetchall()

[('A', 0),
 ('A', 0),
 ('A', 0),
 ('A', 0),
 ('B', 0),
 ('B', 0),
 ('B', 0),
 ('B', 0),
 ('B', 0),
 ('B', 0),
 ('C', 0),
 ('C', 0),
 ('C', 1)]

In [33]:
data.fillna({'month':1})

,product,pop,month
0,A,1,1
1,A,2,2
2,A,1,4
3,A,3,3
4,B,1,1
5,B,2,2
6,B,3,3
7,B,4,4
8,B,3,5
9,B,4,6


In [67]:
engine.execute('select pop1, prod1, coalesce(month, 0) from data').fetchall()

[(1, 'A', 1.0),
 (2, 'A', 2.0),
 (1, 'A', 4.0),
 (3, 'A', 3.0),
 (1, 'B', 1.0),
 (2, 'B', 2.0),
 (3, 'B', 3.0),
 (4, 'B', 4.0),
 (3, 'B', 5.0),
 (4, 'B', 6.0),
 (1, 'C', 1.0),
 (2, 'C', 3.0),
 (3, 'C', 0)]

In [30]:
engine.execute('select pop1, prod1, case when month is null then 100 else month end as nulled_month from data').fetchall()

[(1, 'A', 1.0),
 (2, 'A', 2.0),
 (1, 'A', 4.0),
 (3, 'A', 3.0),
 (1, 'B', 1.0),
 (2, 'B', 2.0),
 (3, 'B', 3.0),
 (4, 'B', 4.0),
 (3, 'B', 5.0),
 (4, 'B', 6.0),
 (1, 'C', 1.0),
 (2, 'C', 3.0),
 (3, 'C', 100)]

#### Get the unique pop and month rows

In [228]:
data.loc[:,['pop','month']].drop_duplicates()

,pop,month
0,1,1
1,2,2
2,1,4
3,3,3
7,4,4
8,3,5
9,4,6
11,2,3
12,3,2


#### Subtract value from columns

In [16]:
data.loc[:,['pop1','month']].sub([1,1],axis='columns')

,pop1,month
0,0,0
1,1,1
2,0,3
3,2,2
4,0,0
5,1,1
6,2,2
7,3,3
8,2,4
9,3,5


In [31]:
engine.execute('select pop1, month - 1 from data').fetchall()

[(1, 0.0),
 (2, 1.0),
 (1, 3.0),
 (3, 2.0),
 (1, 0.0),
 (2, 1.0),
 (3, 2.0),
 (4, 3.0),
 (3, 4.0),
 (4, 5.0),
 (1, 0.0),
 (2, 2.0),
 (3, None)]

In [17]:
data.loc[:,['pop1','month']].div([2,2],axis='columns')

,pop1,month
0,0.5,0.5
1,1.0,1.0
2,0.5,2.0
3,1.5,1.5
4,0.5,0.5
5,1.0,1.0
6,1.5,1.5
7,2.0,2.0
8,1.5,2.5
9,2.0,3.0


#### Data explode

In [35]:
data_explode = pd.DataFrame()
data_explode['A'] = [(1,2,3),(4,5,6)]
data_explode['B'] = ['hi','bi']

In [36]:
data_explode

,A,B
0,"(1, 2, 3)",hi
1,"(4, 5, 6)",bi


In [5]:
data_explode.explode('A')

,A,B
0,1,hi
0,2,hi
0,3,hi
1,4,bi
1,5,bi
1,6,bi


In [8]:
data_explode.groupby('B', sort=False).agg({'A':lambda x: x.values})

,A
B,
hi,"[1, 2, 3]"
bi,"[4, 5, 6]"


## Problems
Email problems I've gone through: 

- 83
- 79 x
- 75
- 39 x
- 38 x
- 35 x


- 32 x
- 31 x
- 30 x
- 29 x
- 28
- 27 x



- 21

- 19
- 18 x
- 17
- 16


- 14: SQL streaming songs
- 10: student attendance
- 7: Bayes: prob of interview
- 5: Employee results - sql
- 1: Store sql

#### Email question 83

In [3]:
jobs = pd.DataFrame()
jobs['name'] = ['John','Harry','Sam','Tina']
jobs['job_role'] = ['Analyst','Administrative','Software','Analyst']


,name,job_role
0,John,Analyst
1,Harry,Administrative
2,Sam,Software
3,Tina,Analyst


In [4]:
jobs.to_sql('jobs',con=engine)

In [9]:
engine.execute('select * from jobs where name like "%a%"').fetchall()

[(1, 'Harry', 'Administrative'),
 (2, 'Sam', 'Software'),
 (3, 'Tina', 'Analyst')]

#### Email question 79

In [40]:

log_activity = pd.DataFrame()
log_activity['user_id'] = [1,1,2, 2,3]
log_activity['date'] = [360,300,350,400,100]

In [42]:
log_activity.to_sql('log_1', con=engine)

In [55]:
engine.execute('select * from log_1 b where not exists (select distinct(a.user_id) from log_1 a where a.date>340 and b.user_id=a.user_id)').fetchall()


[(4, 3, 100)]

In [58]:
engine.execute('select * from log_1 a left join (select distinct(a.user_id), 1 as dude from log_1 a where a.date>340) b on a.user_id=b.user_id ').fetchall()



[(0, 1, 360, 1, 1),
 (1, 1, 300, 1, 1),
 (2, 2, 350, 2, 1),
 (3, 2, 400, 2, 1),
 (4, 3, 100, None, None)]

#### Email question 75

In [3]:
twitch = pd.DataFrame()
twitch['creator_id'] = [1,1,1,1,2,2,2,2,3,3]
twitch['viewer_id'] = [10,10,11,11,12,12,12,13,11,11]
twitch['session_id'] = range(10)
twitch['donation_amt'] = [100,100,200,200,300,100,100,200,400,500]

In [4]:
twitch.to_sql('twitch',con=engine)

In [8]:
engine.execute('select creator_id, avg(amt) as avg_amt from (select creator_id, viewer_id, sum(donation_amt) as amt from twitch group by creator_id, viewer_id) group by creator_id order by avg_amt desc').fetchall()




[(3, 900.0), (2, 350.0), (1, 300.0)]

In [9]:
sum_grouped = twitch.groupby(['creator_id','viewer_id']).agg({'donation_amt':'sum'}).reset_index()

In [12]:
sum_grouped.groupby('creator_id').agg({'donation_amt':'mean'}).reset_index().sort_values('donation_amt',ascending=False)

,creator_id,donation_amt
2,3,900
1,2,350
0,1,300


#### Email question 71

In [14]:
emp_rev = pd.DataFrame()
emp_rev['emp_id'] = [1,2,3,4,5,6]
emp_rev['pa'] = ['A','A','B','B','C','C']

rev = pd.DataFrame()
rev['date'] = [2017,2017, 2017, 2017, 2017, 2018, 2018]
rev['pa'] = ['A','A','B','B','C','C','C']
rev['rev'] = [100,150,200,200,300,100,100]

In [15]:
emp_rev.to_sql('emp_rev', con=engine)
rev.to_sql('rev',con=engine)

In [21]:
engine.execute('select a.pa, b.total_rev/a.emp_count from (select pa, count(distinct(emp_id)) as emp_count from emp_rev group by pa) a inner join (select pa, sum(rev) as total_rev from rev where date=2017 group by pa) b on a.pa=b.pa').fetchall()


[('A', 125), ('B', 200), ('C', 150)]

#### Email question 39

In [68]:
orders = pd.DataFrame()
orders['order_id'] = [1,2,3,4,5,6,7]
orders['month'] = [9,9,10,10,11,8,8]
orders['revenue'] = [100.,125.,200.,80.,200.,90.,100.]

In [69]:
orders.to_sql('orders',con=engine)

In [62]:
engine.execute('select a.month, (a.rev - lag(a.rev, 1)  over ())*1.0/(lag(a.rev,1) over ())*1.0  from (select month, sum(revenue) as rev from orders group by month) a ').fetchall()


[(8, None),
 (9, 0.18421052631578946),
 (10, 0.24444444444444444),
 (11, -0.2857142857142857)]

In [65]:
orders_grouped = orders.groupby('month').agg({'revenue':'sum'}).reset_index()
orders_grouped.sort_values('month',inplace=True, ascending=True)
orders_grouped['rev_growth'] = orders_grouped.revenue.rolling(2).apply(lambda x: (x[1] - x[0])/x[0]).values

/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
orders_grouped

,month,revenue,rev_growth
0,8,190.0,NaN
1,9,225.0,0.184211
2,10,280.0,0.244444
3,11,200.0,-0.285714


#### Email question 38

In [73]:
baby_names = pd.read_csv("https://raw.githubusercontent.com/erood/interviewqs.com_code_snippets/master/Datasets/ddi_baby_names.csv")


In [75]:
baby_names.to_sql('baby',con=engine)

In [76]:
baby_names.head()

,year,name,gender,count
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [78]:
engine.execute('select gender, count(distinct(name)) from baby group by gender').fetchall()

[('F', 67698), ('M', 41475)]

In [3]:
baby_names.groupby('gender').agg({'name':lambda x: x.nunique()})

,name
gender,
F,67698
M,41475


In [99]:
engine.execute('select * from (select gender, name, sum_name, row_number() over (partition by gender order by sum_name desc) as rn from (select gender, name, sum(count) as sum_name from baby group by gender, name) a) where rn>=1 and rn<=10 ').fetchall()



[('F', 'Mary', 4125675, 1),
 ('F', 'Elizabeth', 1638349, 2),
 ('F', 'Patricia', 1572016, 3),
 ('F', 'Jennifer', 1467207, 4),
 ('F', 'Linda', 1452668, 5),
 ('F', 'Barbara', 1434397, 6),
 ('F', 'Margaret', 1248985, 7),
 ('F', 'Susan', 1121703, 8),
 ('F', 'Dorothy', 1107635, 9),
 ('F', 'Sarah', 1077746, 10),
 ('M', 'James', 5164280, 1),
 ('M', 'John', 5124817, 2),
 ('M', 'Robert', 4820129, 3),
 ('M', 'Michael', 4362731, 4),
 ('M', 'William', 4117369, 5),
 ('M', 'David', 3621322, 6),
 ('M', 'Joseph', 2613304, 7),
 ('M', 'Richard', 2565301, 8),
 ('M', 'Charles', 2392779, 9),
 ('M', 'Thomas', 2311849, 10)]

In [12]:
baby_names = baby_names.rename(columns = {'count':'total'})
baby_popular = baby_names.groupby(['name','gender']).agg({'total':'sum'}).reset_index()
baby_popular_men = baby_popular[baby_popular.gender=='M'].sort_values('total', ascending=False).iloc[:,:10]
baby_popular_women = baby_popular[baby_popular.gender=='F'].sort_values('total', ascending=False).iloc[:,:10]

In [26]:
baby_names_popular = baby_names.groupby(['name','gender']).agg({'count':'sum'}).reset_index()


In [29]:
baby_names_popular.groupby('gender').apply(lambda x: x.sort_values('count',ascending=False).iloc[:10])

name gender    count
gender                                  
F      68679        Mary      F  4125675
       30706   Elizabeth      F  1638349
       79184    Patricia      F  1572016
       46567    Jennifer      F  1467207
       62973       Linda      F  1452668
       11699     Barbara      F  1434397
       67371    Margaret      F  1248985
       94540       Susan      F  1121703
       28575     Dorothy      F  1107635
       87546       Sarah      F  1077746
M      43582       James      M  5164280
       48388        John      M  5124817
       84427      Robert      M  4820129
       71023     Michael      M  4362731
       104615    William      M  4117369
       24204       David      M  3621322
       49192      Joseph      M  2613304
       83864     Richard      M  2565301
       18299     Charles      M  2392779
       98544      Thomas      M  2311849

In [27]:
baby_names_2010 = baby_names[baby_names.year>2010]

In [29]:
baby_names_2010 = baby_names_2010.groupby('name').filter(lambda x: x.shape[0]==8)

In [35]:
# baby_names_2010 = baby_names_2010.rename(columns={'count':'total'})
baby_names_2010 = baby_names_2010.groupby('name', sort=False).apply(lambda x: x.loc[:,['year','total']].sort_values('year',ascending=True)).reset_index()

# (total.rolling(2).apply(lambda x: (x[1] - x[0])/x[0]).values


In [40]:
baby_names_2010['per_growth'] = baby_names_2010.groupby('name', sort=False).total.rolling(2).apply(lambda x: (x[1] - x[0])/x[0]).values

/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [44]:
baby_names_2010 = baby_names_2010.groupby('name').filter(lambda x: x.shape[0]==2)

In [45]:
baby_names_2010 = baby_names_2010.groupby('name').apply(lambda x: x.loc[:,['year','count']].sort_values('year', ascending=True)).reset_index()

In [51]:
baby_names_2010.columns = ['name','level_1','year','count_1']

In [54]:
baby_names_2010['gain'] = baby_names_2010.groupby('name').count_1.rolling(2).apply(lambda x: (x[1] - x[0])/x[0]).values


/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [55]:
baby_names_2010

,name,level_1,year,count_1,gain
0,Aaban,1685983,2010,9,NaN
1,Aaban,1952653,2018,7,-0.222222
2,Aadam,1687535,2010,7,NaN
3,Aadam,1947920,2018,19,1.714286
4,Aaden,1678414,2010,450,NaN
...,...,...,...,...,...
38489,Zyriah,1930763,2018,22,-0.241379
38490,Zyron,1687534,2010,8,NaN
38491,Zyron,1957044,2018,5,-0.375000
38492,Zzyzx,1692111,2010,5,NaN


#### Email Problem 35 - Sales Analyst

In [100]:
all_sales = pd.DataFrame()
all_sales['sales_rep_id'] = [1,1,2,2,3]
all_sales['date'] = [100,101,205,200,150]
sales_rep_info = pd.DataFrame()
sales_rep_info['sales_rep_id'] = [1,2,3]
sales_rep_info['date_hired'] = [4,6,8]

In [101]:
all_sales.to_sql('all_sales',con=engine)

In [102]:
sales_rep_info.to_sql("sales_rep", con=engine)

In [31]:
engine.execute("select a.sales_rep_id, min(a.date - b.date_hired)  from all_sales a inner join sales_rep b on a.sales_rep_id=b.sales_rep_id group by a.sales_rep_id  ").fetchall()

[(1, 96), (2, 194), (3, 142)]

In [16]:
comb_info = all_sales.merge(sales_rep_info,on='sales_rep_id')
comb_info.groupby('sales_rep_id').apply(lambda x: min(x['date'] - x['date_hired']))

sales_rep_id
1     96
2    194
3    142
dtype: int64

In [45]:
first_sale = all_sales.groupby('sales_rep_id').agg({'date':'min'}).reset_index()
first_sale_rep = first_sale.merge(sales_rep_info,on='sales_rep_id')
first_sale_rep['how_long'] = first_sale_rep['date'] - first_sale_rep['date_hired']


In [7]:
from datetime import datetime

In [11]:
m = datetime.strptime('2017-10-01','%Y-%m-%d')
m1 = datetime.strptime('2017-11-01','%Y-%m-%d')

In [15]:
type(m.month - m1.month)

int

#### Email Question 32

In [108]:
m = [1, 2, 3, 1, 4, 5, 2, 3, 6]
A = 3

In [109]:
for i in range(len(m) - (A-1)):
    print(max(m[i:i+A]))

3
3
4
5
5
5
6


In [11]:
for i in range(len(m)):
    if (i+3)>len(m):
        break
    else:
        print(max(m[i:i+3]))

3
3
4
5
5
5
6


In [111]:
for i,j in [(1,2),(3,4)]:
    print(i)
    print(j)

1
2
3
4


In [ ]:
2+2

In [113]:
"s3://dlx-prod-analytics/FE/omni_propensity/{0}/omni_propensity_{1}/"\
                                              "fulfill_info_dictionary/category={2}/"\
                                              "{2}_fulfill_info.json"

's3://dlx-prod-analytics/FE/omni_propensity/{0}/omni_propensity_{1}/fulfill_info_dictionary/category={2}/{2}_fulfill_info.json'

#### Email problem 31 - year over year raises

In [87]:
employee = pd.DataFrame()
employee['salary'] = [80000,70000,60000,59000,65000,60000,65000,60000]
employee['year'] = [2020,2019,2018,2017,2019,2018,2019,2018]
employee['emp_id'] = [1,1,1,1,2,2,3,3]

employee.to_sql('employee',con=engine)

In [106]:
engine.execute('select * from (select emp_id, sum(bool_raise) over (partition by emp_id order by year asc rows 2 preceding) as in_row from (select emp_id, year, salary>lag(salary,1) over (partition by emp_id order by year asc) as bool_raise from employee) a) b where b.in_row>=3 ').fetchall()



[(1, 3)]

In [115]:
employee.sort_values(['emp_id', 'year'], ascending=True, inplace=True)
employee['diffs'] = employee.groupby('emp_id', sort=False).salary.rolling(2).apply(lambda x: (x[1] - x[0])>0).values

/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  


In [117]:
employee.groupby('emp_id').diffs.rolling(3).apply(lambda x: sum(x))

/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


emp_id   
1       3    NaN
        2    NaN
        1    NaN
        0    3.0
2       5    NaN
        4    NaN
3       7    NaN
        6    NaN
Name: diffs, dtype: float64

In [48]:
employee = employee.groupby('emp_id').apply(lambda x: x.loc[:,['year','salary']].sort_values('year',ascending=True)).reset_index()

In [49]:
employee['increase'] = employee.groupby('emp_id', sort=False).salary.rolling(2).apply(lambda x: (x[1] - x[0])>0).values

/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [50]:
employee['three_in_row'] = employee.groupby('emp_id', sort=False).increase.rolling(3).apply(lambda x: sum(x)==3).values

/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [51]:
employee[employee.three_in_row==True].emp_id.unique()

array([1])

In [52]:
employee

,emp_id,level_1,year,salary,increase,three_in_row
0,1,3,2017,59000,NaN,NaN
1,1,2,2018,60000,1.0,NaN
2,1,1,2019,70000,1.0,NaN
3,1,0,2020,80000,1.0,1.0
4,2,5,2018,60000,NaN,NaN
5,2,4,2019,65000,1.0,NaN
6,3,7,2018,60000,NaN,NaN
7,3,6,2019,65000,1.0,NaN


In [56]:
employee.sort_values(['emp_id','year'], ascending=False, inplace=True)

In [57]:
employee['sal_diff'] = employee.groupby('emp_id', sort=False).salary.diff()

In [58]:
employee['sal_diff_sign'] = employee['sal_diff']<0

In [59]:
employee['rolling'] = employee.groupby('emp_id', sort=False).sal_diff_sign.rolling(3).sum().values

In [60]:
employee

,salary,year,emp_id,sal_diff,sal_diff_sign,rolling
6,65000,2019,3,NaN,False,NaN
7,60000,2018,3,-5000.0,True,NaN
4,65000,2019,2,NaN,False,NaN
5,60000,2018,2,-5000.0,True,NaN
0,80000,2020,1,NaN,False,NaN
1,70000,2019,1,-10000.0,True,NaN
2,60000,2018,1,-10000.0,True,2.0
3,59000,2017,1,-1000.0,True,3.0


#### Email question 29 - ways of getting a football score

In [63]:
import math
def diff_ways(score):
    save_combo = []
    a_max = math.floor(score/2.0)
    b_max = math.floor(score/3.0)
    c_max = math.floor(score/6.0)
    d_max = math.floor(score/7.0)
    e_max = math.floor(score/8.0)
    
    for a in range(a_max + 1):
        for b in range(b_max + 1):
            for c in range(c_max + 1):
                for d in range(d_max + 1):
                    for e in range(e_max + 1):
                        if 2*a + 3*b + 6*c + 7*d + 8*e==score:
                            save_combo.append([a,b,c,d,e])
                        else:
                            pass
    return(save_combo)
        

In [ ]:
def diff_ways(score):
    for i in [2.0,3.0,6.0,7.0,8.0]:
        if score%i==0:
            diff_ways(score/i)
        else:
            return score/i

#### Email practice 27

In [3]:
airbnb = pd.DataFrame()
airbnb['property_id'] = [1,2,3,4]
airbnb['country'] = ['A','A','B','C']
airbnb['city_name'] = ['Boston','Boston','Berlin','Paris']
airbnb['subregion_name'] = ['MA','WY','Frank','Louve']
airbnb['addres'] = ['123','345','567','789']

In [7]:
airbnb.to_sql('airbnb',con=engine)

In [4]:
guest = pd.DataFrame()
guest['guest_id'] = [123,345,123,678]
guest['property_id'] = [1,1,2,4]
guest['date_start'] = ['2017-07-10','2017-10-02','2017-12-28','2017-01-03']
guest['date_end'] = ['2017-07-12','2017-10-05','2017-01-01','2017-01-05']
guest['stay_length'] = [2,3,4,2]
guest['airbnb_revenue'] = [200,300,400,100]

In [5]:
guest['year'] = guest.date_start.apply(lambda x: int(x[:4]))

In [6]:
guest.to_sql('guest',con=engine)

In [9]:
engine.execute("select c.property_id, sum(c.airbnb_revenue) as sum_rev, c.city_name, c.country, c.subregion_name from (select g.property_id, g.airbnb_revenue, a.city_name, a.country, a.subregion_name from guest g inner join airbnb a on g.property_id=a.property_id where g.year=2017) c group by c.property_id, c.city_name, c.country, c.subregion_name order by sum_rev desc  ").fetchall()



[(1, 500, 'Boston', 'A', 'MA'),
 (2, 400, 'Boston', 'A', 'WY'),
 (4, 100, 'Paris', 'C', 'Louve')]

In [78]:
guest_2017 = guest[guest.year==2017]

In [79]:
guest_2017_property = guest_2017.merge(airbnb,on='property_id')

In [80]:
guest_2017_property.groupby(['property_id','city_name','country','subregion_name']).agg({'airbnb_revenue':'sum'})

,,,,airbnb_revenue
property_id,city_name,country,subregion_name,
1,Boston,A,MA,500
2,Boston,A,WY,400
4,Paris,C,Louve,100


In [81]:
# filter to year
guest['year_end'] = guest.date_end.apply(lambda x: int(x[:4]))
guest['year_beg'] = guest.date_start.apply(lambda x: int(x[:4]))
guest = guest[(guest.year_end==2017) & (guest.year_beg==2017)]


property_table = airbnb.merge(guest, on='property_id')
revenue = property_table.groupby(['country','city_name','subregion_name']).agg({'airbnb_revenue':'sum'}).reset_index()
revenue.columns = ['country','city_name','subregion_name','summed_rev']

In [82]:
guest

,guest_id,property_id,date_start,date_end,stay_length,airbnb_revenue,year_end,year_beg
0,123,1,2017-07-10,2017-07-12,2,200,2017,2017
1,345,1,2017-10-02,2017-10-05,3,300,2017,2017
2,123,2,2017-12-28,2017-01-01,4,400,2017,2017
3,678,4,2017-01-03,2017-01-05,2,100,2017,2017


In [83]:
airbnb

,property_id,country,city_name,subregion_name,addres
0,1,A,Boston,MA,123
1,2,A,Boston,WY,345
2,3,B,Berlin,Frank,567
3,4,C,Paris,Louve,789


In [84]:
property_table

,property_id,country,city_name,subregion_name,addres,guest_id,date_start,date_end,stay_length,airbnb_revenue,year_end,year_beg
0,1,A,Boston,MA,123,123,2017-07-10,2017-07-12,2,200,2017,2017
1,1,A,Boston,MA,123,345,2017-10-02,2017-10-05,3,300,2017,2017
2,2,A,Boston,WY,345,123,2017-12-28,2017-01-01,4,400,2017,2017
3,4,C,Paris,Louve,789,678,2017-01-03,2017-01-05,2,100,2017,2017


In [85]:
revenue

,country,city_name,subregion_name,summed_rev
0,A,Boston,MA,500
1,A,Boston,WY,400
2,C,Paris,Louve,100


#### Email question 22 - get sample of users from each group

In [64]:
user_devices = pd.DataFrame()
user_devices['user_id'] = [1,2,3,4,5,6,7,8]
user_devices['devices'] = [['watch','computer'],['watch','phone'],['watch','phone','computer'],
                           ['watch','phone','computer'],['watch','phone','computer'],
                          ['watch','phone','computer'],['watch','phone','computer'],
                          ['watch','computer']]

In [65]:
user_devices = user_devices.explode('devices')
watch_df = user_devices[user_devices.devices=='watch']
phone_df = user_devices[user_devices.devices=='phone']
computer_df = user_devices[user_devices.devices=='computer']

# watch_sample = watch_df.sample(2)
# watch_sample.columns = ['user_id','device_watch']
# phone_watch_merge = phone_df.merge(watch_sample, on='user_id',how='left')
# phone_watch_merge = phone_watch_merge[phone_watch_merge.device_watch.isna()]
# phone_sample = phone_watch_merge.sample(2)

# selected_users = pd.concat([phone_sample,watch_sample], sort=False)
# selected_users['selected'] = 1
# computer_watch_phone_merge = computer_df.merge(selected_users, on='user_id',how='left')
# computer_sample = computer_watch_phone_merge[computer_watch_phone_merge.selected.isna()].sample(2)



In [66]:
watch_sample = watch_df.sample(2)

In [67]:
remaining_users = set(user_devices.user_id.values).difference(set(watch_sample.user_id.values))

In [70]:
remain_users_df = pd.DataFrame()
remain_users_df['user_id'] = np.array(remaining_users)
# phone_df.merge(remain_users_df, on='user_id')

#### EMAIL Problem #21: Spread a months aggregated amount across days in the month

In [6]:
month_agg = pd.DataFrame()
month_agg['month'] = [1,2,3]
month_agg['rev'] = [300,330,390]

In [7]:
month_agg

,month,rev
0,1,300
1,2,330
2,3,390


In [8]:
for i in month_agg.month.unique():
    month_agg = month_agg.append([month_agg[month_agg.month==i]]*29, ignore_index=True)


In [20]:
month_agg['dist'] = month_agg.groupby('month', sort=False).rev.apply(lambda x: x/30.0)

In [23]:
month_agg = month_agg.sort_values('month')

In [27]:
month_agg['day'] = [i for i in range(1,31)]*3

Alternative solution

In [36]:
daily_agg = pd.DataFrame()
daily_agg['month'] = month_agg.month.unique().repeat(30)

In [37]:
daily_agg['days'] = [i for i in range(1,31)]*3

In [38]:
daily_agg['totals'] = month_agg.rev.unique().repeat(30)

In [39]:
daily_agg['daily_total'] = daily_agg['totals']/30.0

In [40]:
daily_agg

,month,days,totals,daily_total
0,1,1,300,10.0
1,1,2,300,10.0
2,1,3,300,10.0
3,1,4,300,10.0
4,1,5,300,10.0
...,...,...,...,...
85,3,26,390,13.0
86,3,27,390,13.0
87,3,28,390,13.0
88,3,29,390,13.0


#### Email question 18 - percent of revenue coming from loyal customers

In [7]:
hotels = pd.DataFrame()
hotels['customer_id'] = [1,1,1,1,1,1,1,1,1,1,2,2,3,3,3,3]
hotels['hotel_id'] = [1,1,1,1,2,2,2,3,3,3,2,4,1,3,2,4]
hotels['transaction_id'] = range(16)
hotels['total_spend'] = [100]*16
hotels['is_member'] = [True, True, True,True, True, True, True,True, True,True, False, False, True, True, True, True]

In [8]:
hotels.to_sql('hotels',con=engine)

In [22]:
engine.execute('select 1.0*s/(select sum(total_spend) from hotels) from (select customer_id, count(hotel_id) a , sum(count_tran) b, sum(spend) as s from (select customer_id, hotel_id, count(transaction_id)>=2 as count_tran, sum(total_spend) spend from hotels where is_member is True group by customer_id, hotel_id) group by customer_id having a>=3 and a=b)').fetchall()


[(0.625,)]

In [13]:
engine.execute('select customer_id, hotel_id, count(transaction_id) as num_hotel from hotels h where h.is_member is True group by customer_id, hotel_id having num_hotel>2  ').fetchall()



[(1, 1, 4), (1, 2, 3), (1, 3, 3)]

In [42]:
member = hotels[hotels.is_member==True]
member = member.groupby('customer_id').filter(lambda x: x['hotel_id'].nunique()>=3)
member_transaction = member.groupby(['customer_id','hotel_id']).agg({'transaction_id':'count','total_spend':'sum'}).reset_index()
member_transaction = member_transaction.groupby('customer_id').filter(lambda x: sum(x['transaction_id']>2)==x.shape[0])

# loyal_customers.total_spend.sum()/hotels.total_spend.sum()





In [44]:
member_transaction.total_spend.sum()/hotels.total_spend.sum()

0.625

In [48]:
member_transaction

,customer_id,hotel_id,transaction_id,total_spend
0,1,1,4,400
1,1,2,3,300
2,1,3,3,300


In [53]:
2+2

4

In [50]:
points_member = hotels[hotels.is_member==True]
at_least_3 = points_member.groupby('customer_id').filter(lambda x: x['hotel_id'].nunique()>=3)
greater_2 = at_least_3.groupby(['customer_id','hotel_id']).agg({'transaction_id':lambda x: x.nunique(), 'total_spend':'sum'}).reset_index()
greater_2['bool_greater'] = greater_2.groupby('customer_id').transaction_id.transform(lambda x: x>2).values

In [51]:
final_df = greater_2.groupby('customer_id').agg({'bool_greater':['count','sum'], 'total_spend':'sum'}).reset_index()
final_df.columns = ['customer_id','bool_count','bool_sum','rev']
final_df[final_df.bool_count==final_df.bool_sum]

,customer_id,bool_count,bool_sum,rev
0,1,3,3.0,1000


#### Email question 17

In [41]:
df = pd.DataFrame()
df['employee_id'] = ['1','2','3','4']
df['yrs_at_company'] = [6,20,8,20]
df['compensation'] = [1000,2000,1500,2500]

In [42]:
df

,employee_id,yrs_at_company,compensation
0,1,6,1000
1,2,20,2000
2,3,8,1500
3,4,20,2500


In [46]:
def case_statement(x,max_tenure):
    for i in range(0,max_tenure,5):
        if (x>i) and (x<=(i+5)):
            return '{0}-{1}'.format(i,i+5)

df['buckets'] = df.yrs_at_company.apply(case_statement, args=[20])

In [47]:
df

,employee_id,yrs_at_company,compensation,buckets
0,1,6,1000,5-10
1,2,20,2000,15-20
2,3,8,1500,5-10
3,4,20,2500,15-20


In [52]:
df.groupby('buckets',sort=False).agg({'compensation':['max','min','mean','median','std','var']})

compensation                                       
                 max   min  mean median         std     var
buckets                                                    
5-10            1500  1000  1250   1250  353.553391  125000
15-20           2500  2000  2250   2250  353.553391  125000

#### Email Question 16 - overlapping intervals

In [26]:
P =  [[0, 2], [3, 7], [4, 6], [7, 8], [1 ,5]]
z = 5

def overlapping(z,P):
    for i in range(len(P)):
        if (z>=P[i][0]) and (z<=P[i][1]):
            print(P[i])

overlapping(z,P)

[3, 7]
[4, 6]
[1, 5]


In [89]:
P =  [[0, 2], [3, 7], [4, 6], [7, 8], [1 ,5]]
z = 5

def overlapping(P, z):
    t = [z in set(range(i[0], i[1]+1)) for i in P]
    t = [True if (z>= i[0]) and (z<=i[1]) else False for i in P]
    print(sum(t))
    print(np.array(P)[t])
    
    for i in range(len(P)):
        if t[i]==True:
            print(P[i])

    

In [90]:
overlapping(P,z)

3
[[3 7]
 [4 6]
 [1 5]]
[3, 7]
[4, 6]
[1, 5]


#### Email question 14 - streaming service, average number of hours a user spends listening to music daily

user id
time started listening
song id
artist id

song_id
artist_id
song_length

In [10]:
user = pd.DataFrame()
user['user_id'] = [1,1,2,3,3,3,4,4]
user['time'] = [200,300,100,150,200,400,400,350]
user['song_id'] = [1,2,1,3,4,2,1,2]
user['artist_id'] = ['a','b','a','c','d','b','a','b']

song = pd.DataFrame()
song['song_id'] = [1,2,3,4]
song['artist_id'] = ['a','b','c','d']
song['dur'] = [2,3,2,4]


In [11]:
user.to_sql('user',con=engine)

In [12]:
song.to_sql('song',con=engine)

In [17]:
engine.execute('select c.user_id, avg(c.total_dur) from (select a.user_id, sum(b.dur) as total_dur from user a inner join song b on a.song_id=b.song_id group by a.user_id, a.time) c group by c.user_id').fetchall()

[(1, 2.5), (2, 2.0), (3, 3.0), (4, 2.5)]

In [19]:
merged_data = user.merge(song, on='song_id', how='inner')
merged_data = merged_data.groupby(['user_id','time']).agg({'dur':'sum'}).reset_index()
merged_data.groupby('user_id').agg({'dur':'mean'})

,dur
user_id,
1,2.5
2,2.0
3,3.0
4,2.5


In [36]:
def convert_to_day(x):
    if x<=200:
        return(1)
    elif (x>200) & (x<=300):
        return(2)
    else:
        return(3)

user_song_length = user.merge(song, on=['song_id','artist_id'])
# user_song_length['day'] = user_song_length.time.apply(convert_to_day)
user_song_length = user_song_length.groupby(['user_id','time']).agg({'dur':'sum'}).reset_index()
user_song_length = user_song_length.groupby('user_id').agg({'dur':'mean'}).reset_index()
user_song_length['dur'].mean()/60.0


0.041666666666666664

In [37]:
user_song_length

,user_id,dur
0,1,2.5
1,2,2.0
2,3,3.0
3,4,2.5


#### Email question 10: student attendance

In [38]:
student_attendance_log = pd.DataFrame()
student_attendance_log['student_id'] = [1,1,2,3,4]
student_attendance_log['date'] = ['2018-03-12','2019-03-12','2018-03-12','2018-03-12','2018-03-12']
student_attendance_log['attendance_status'] = ['tardy','tardy','absent','present','present']

student_demographics = pd.DataFrame()
student_demographics['student_id'] = [1,2,3,4]
student_demographics['grade'] = [6,6,7,7]


In [39]:
student_attendance_log.to_sql('att_log',con=engine)
student_demographics.to_sql('demo',con=engine)

In [43]:
engine.execute('select 1.0*sum(att_bool)/count(student_id) from (select *, case when attendance_status="absent" then 1 else 0 end as att_bool from att_log a inner join demo d on a.student_id=d.student_id where a.date="2018-03-12") group by grade  ').fetchall()



[(0.5,), (0.0,)]

In [44]:
student_attendance_03122018 = student_attendance_log[student_attendance_log.date=='2018-03-12']
student_attendance_03122018['attended'] = student_attendance_03122018.attendance_status.apply(lambda x: True if x!='absent' else False)
# student_attendance_03122018['attended'] = student_attendance_03122018.attendance_status!='absent'
student_join = student_attendance_03122018.merge(student_demographics, on='student_id')
final_answer = student_join.groupby('grade').agg({'attended':'mean'})



/Users/magoodwi/MyStuff/Education/DataScienceToolkit/DataScienceCode/ds_python_37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
final_answer

,attended
grade,
6,0.5
7,1.0


#### Email question 5

In [46]:
survey_logging = pd.DataFrame()
survey_logging['employee_id'] = [1,1,2,2,3,3,4,4]
survey_logging['question_id'] = [1,1,1,1,2,2,2,2]
survey_logging['response'] = ['view', 'answer','view','ignore','view','answer','view','answer']

In [47]:
survey_logging.to_sql('survey_logging',con=engine)

In [52]:
engine.execute('select question_id, sum(resp_bool)*1.0/count(response) from (select question_id, response, case when response="answer" then 1 else 0 end as resp_bool from survey_logging) group by question_id').fetchall()



[(1, 0.25), (2, 0.5)]

In [53]:
survey_logging['view_bool'] = survey_logging.response.apply(lambda x: True if x=='view' else False).values
survey_logging['answer_bool'] = survey_logging.response.apply(lambda x: True if x=='answer' else False).values
survey_questions = survey_logging.groupby('question_id').agg({'view_bool':'sum','answer_bool':'sum'}).reset_index()
survey_questions['response_rate'] = survey_questions['answer_bool']/survey_questions['view_bool']

In [54]:
survey_questions

,question_id,view_bool,answer_bool,response_rate
0,1,2.0,1.0,0.5
1,2,2.0,2.0,1.0


In [77]:
survey_questions[survey_questions.response_rate==max(survey_questions.response_rate)]

,question_id,view_bool,answer_bool,response_rate
1,2,2.0,2.0,1.0


In [86]:
resp_rate = survey_logging.groupby('question_id').apply(lambda x: sum(x['response']=='answer')/sum(x['response']=='view')).reset_index()



,question_id,0
0,1,0.5
1,2,1.0


#### Email question 4

In [57]:
user_messaging = pd.DataFrame()
user_messaging['sender_id'] = [1,1,1,1,2,2,3,3,3,3]
user_messaging['reciever_id'] = [4,5,6,7,8,9,10,11,12,13]

In [60]:
user_messaging.groupby('sender_id').filter(lambda x: x['reciever_id'].nunique()>3).sender_id.nunique()/user_messaging.sender_id.nunique()

0.6666666666666666

#### Email question 2

In [16]:
import numpy as np

In [25]:
# Array of numbers
J = [4, 4, 4, 9, 10, 11, 12]
# Length of sequences, p
p = 3

def min_max_avg(J,p):
    max_mean = -np.inf
    min_mean = np.inf
    for i in range(len(J)-2):
        print(J[i:i+p])
        seq_mean = np.mean(J[i:i+p])

        if seq_mean > max_mean:
            max_mean = seq_mean
        if seq_mean < min_mean:
            min_mean = seq_mean
    return([max_mean, min_mean]) 

In [26]:
min_max_avg(J,p)

[4, 4, 4]
[4, 4, 9]
[4, 9, 10]
[9, 10, 11]
[10, 11, 12]


[11.0, 4.0]

#### Email question 1 - fraudulant stores

In [5]:
store_account = pd.DataFrame()
store_account['store_id'] = [1,2,3,1,2,3,1,2,3]
store_account['revenue'] = [100,1,1,10,10,0, 0,10,0]
store_account['status'] = ['open','open','open','fraud', 'open', 'closed','fraud','open','closed']
store_account['date'] = [1,1,1,2,2,2,3,3,3]

In [6]:
store_account = store_account[store_account.revenue>0]
store_account['fraud_bool'] = store_account.status.apply(lambda x: True if x=='fraud' else False)
store_account.groupby('date').agg({'fraud_bool':'mean'})


,fraud_bool
date,
1,0.0
2,0.5
3,0.0


## Other questions

In [2]:
n = int(3)

arr = list(map(int, '-4 3 -9 0 4 1'.rstrip().split()))


In [11]:
round(len([i for i in arr if i>0])*1000/len(arr),6)s

0.5

In [57]:
from collections import Counter

In [62]:
[i if i>1 else 0 for i in range(10)]

[0, 0, 2, 3, 4, 5, 6, 7, 8, 9]

#### Problem: Query a list of alphabetic names followed by the first letter of their profession

In [27]:
occupation = pd.DataFrame()
occupation['name'] = ['Sarah','Mike','Matt','Kenzie']
occupation['occupation'] = ['Doctor','Actor','Singer','Singer']

In [28]:
occupation.sort_values('name',inplace=True)

In [29]:
occupation['abb'] = occupation['occupation'].apply(lambda x: ' (' + x[0] + ')')
occupation['name_occup'] = occupation['name'] + occupation['abb']

In [30]:
occupation

,name,occupation,abb,name_occup
3,Kenzie,Singer,(S),Kenzie (S)
2,Matt,Singer,(S),Matt (S)
1,Mike,Actor,(A),Mike (A)
0,Sarah,Doctor,(D),Sarah (D)


#### Problem: Query the number of occurences of each occupation and sort by ascending order

In [39]:
occupation_group = occupation.groupby('occupation').agg({'name':'count'}).reset_index()
occupation_group.columns = ['occupation','occupation_count']
occupation_group.sort_values(['occupation_count','occupation'],inplace=True)

#### Problem: Which products had at least a 3 month consecutive increase in their popularity?

In [50]:
data.groupby('prod1',sort=False)

pandas.core.series.Series

In [37]:
data.sort_values(['prod1','month'], ascending=False, inplace=True)

In [38]:
data['diff'] = data.groupby('prod1').pop1.diff()

In [40]:
data['diff_sign'] = data['diff']<0

In [206]:
data['diff_sum'] = data.groupby('prod1',sort=False).diff_sign.rolling(3).sum().values

In [212]:
data.loc[data.diff_sum>=3,'product'].values

array(['B'], dtype=object)

In [35]:
num = 0
ar = [1, 2, 2, 6, 1, 2]
n = 6
k = 3
for i in range(n-1,0,-1):
    val = ar[i]
    print(val)
    print([((val + j)%k)==0 for j in ar[:i]])
    num += sum([((val + j)%k)==0 for j in ar[:i]])

2
[True, False, False, False, True]
1
[False, True, True, False]
6
[False, False, False]
2
[True, False]
2
[True]


In [32]:
for i in range(4,1,-1):
    print(i)

4
3
2


#### One food company interview I had

In [22]:
movies = pd.DataFrame()
movies['movie_id'] = [1,2,3,4]
movies['title'] = ['Gone','with','the','wind']

reviewers = pd.DataFrame()
reviewers['name'] = ['Bob','Nancy','Themla']
reviewers['rev_id'] = [1,2,3]

reviews = pd.DataFrame()
reviews['movie_id'] = [1,1,2,2,2,3,3,4]
reviews['rev_id'] = [1,2,1,2,3,2,3,1]
reviews['stars'] = [3,2,4,5,1,2,3,4]

In [23]:
movies.to_sql('movies', con=engine)
reviewers.to_sql('reviewers', con=engine)
reviews.to_sql('reviews', con=engine)

In [30]:
# What is the average stars Bob would give across movies?
engine.execute('select * from (select b.name, avg(a.stars) from reviews a inner join reviewers b on a.rev_id=b.rev_id group by b.name) c where c.name="Bob"').fetchall()


[('Bob', 3.6666666666666665)]

In [32]:
# Who reviewed the movie "Gone"?
engine.execute('select c.name, d.title from reviewers c inner join (select * from reviews a inner join movies b on a.movie_id=b.movie_id where b.title="Gone") d on c.rev_id=d.rev_id').fetchall()


[('Bob', 'Gone'), ('Nancy', 'Gone')]

## Project Euler

1.

In [6]:
def multiple_sum(n):
    sumit = 0
    for i in range(n):
        if i%3==0 or i%5==0:
            sumit += i
    return(sumit)

In [8]:
multiple_sum(1000)

233168

2.

In [45]:
def sum_even_fib(n):
    seq = [1,2]
    even = [2]
    keep_going = True
    while keep_going==True:
        next_fib = sum(seq[-2:])
        if next_fib>n:
            keep_going=False
        else:
            if (next_fib%2)==0:
                even.append(next_fib)
            seq.append(next_fib)
    return sum(even)

In [46]:
sum_even_fib(4000000)

4613732

3.

In [3]:
def check_prime(n):
    prime=True
    for i in range(2, round(n**.5)+1):
        if n%i==0:
            prime=False
    return prime
        

In [80]:
def largest_prime_factor(n):
    keep_going = True
    cur_prime = 2
    prime_factor = []
    # check if prime
    while keep_going==True:
        if (n/cur_prime)%1==0:
            prime_factor.append(cur_prime)
            n = n/cur_prime
            if n==1:
                keep_going=False
            cur_prime = 2
        else:
            next_prime=True
            while next_prime==True:
                cur_prime += 1
                if check_prime(cur_prime)==True:
                    next_prime=False
    return max(prime_factor)
                    
                

In [82]:
largest_prime_factor(600851475143)

6857

In [51]:
def largest_prime_factor(n):
    cur_prime = 2
    prime_factor = []
    cont=True
    while cont==True:
        
        # check if prime
        if (n/cur_prime)%1==0:
            prime_factor.append(cur_prime)
            n = n/cur_prime
            if n==1:
                cont=False
            else:
                prime_factor = prime_factor + largest_prime_factor(n)
                cont=False
        else:
            next_prime=True
            while next_prime==True:
                cur_prime += 1
                if check_prime(cur_prime)==True:
                    next_prime=False

    return(prime_factor)
        
                    

In [52]:
largest_prime_factor(600851475143)

[71, 839, 1471, 6857]

4.

In [48]:
def palindrome(x):
    x = str(x)
    x_split = [x[i] for i in range(len(x))]
    if len(x_split)%2==0:
        m = x_split[int(len(x_split)/2):]
        m.reverse()
        return(x_split[:int(len(x_split)/2)]==m)
    else:
        m = x_split[int(len(x_split)/2)+1:]
        m.reverse()
        return(x_split[:int(len(x_split)/2)]==m)

In [50]:
palindrome(90509)

True

In [53]:
done=False
max_value = 0
for i in range(999,99,-1):
    for j in range(999,99,-1):
        if palindrome(i*j):
            if (i*j)>max_value:
                max_value = i*j


In [54]:
max_value

906609

Hacker Rank

In [42]:
q.pop(5)

3

In [46]:
q.insert(2,3)

In [49]:
q

[1, 2, 3, 5, 2, 4, 6]

In [65]:
q = [2, 5, 1, 3, 4]
m = [i + 1 for i in range(len(q))]
count = 0
for i in range(len(q)):
    if (q[i] - 1) - q.index(q[i]) > 2:
        print("Too Chaotic")
        break
    if q[i]!=m[i]:
        if (q[i] - m[i])>0:
            count += (m.index(q[i]) - q.index(q[i]))
        m.pop(m.index(q[i]))
        m.insert(i, q[i])

        
    
#     if (q[i] - 1) - q.index(q[i]) > 2:
#         print("Too Chaotic")
#         break
#     else:
#         count.append((q[i] - 1) - q.index(q[i]))
        

        

Too Chaotic


In [64]:
count

3

#### Amazon Interview

In [ ]:
product = pd.DataFrame()
product['product_id'] = [10001,10002, 10003]
product['category'] = ['A','A','B']


Table 2: customer order table (5 years data)
customer_id order_id credit_card_id product_id
C1027       736282   37362 10001
C1625       478e6    4857 10026
C1027       736282    37362 10001
C1027       736282   363 100035

Question

Q1. add a column to table 1, as Grocery_yesno, when a category is A, X, Y, P then the product is a grocery,
otherwise not indicating the purchasing product as grocery or not
product_table['Grocery_yesno'] = product_table.category.apply(lambda x: if x in ['A','X','Y','P'] True else False)


2. add a column to table 2, as primaryCard_yesno, when credit_card_id is the primary card then yes else no
grouped_customer = customer_order_table.groupby(['customer_id','credit_card_id']).agg({'order_id':'count'}).reset_index()
grouped_customer_2 = grouped_customer.sort_values(['customer_id','order_id'],ascending=False).groupby('customer_id').apply(lambda x: x.iloc[0,:]).reset_index()
joined_customer_table = customer_order_table.merge(grouped_customer_2, on='customer_id')
joined_customer_table['primaryCard_yesno'] = joined_customer_table['credit_card_id']==joined_customer_table['credit_card_id_y']


3. merge table 1 or 2 to bring the product information into the order table
product_table.merge(joined_customer_table, on='product_id')

In [34]:
product_table = pd.DataFrame()
product_table['product_id'] = [10001,10002, 10003]
product_table['category'] = ['A','A','B']

In [35]:
customer_order_table = pd.DataFrame()
customer_order_table['customer_id'] = ['C1027','C1625','C1027','C1027']
customer_order_table['order_id'] = [1,2,3,4]
customer_order_table['credit_card_id'] = [37362, 4857, 37362, 363]
customer_order_table['product_id'] = [10001,10026,10001,100035]

In [37]:
product_table.to_sql('prod',con=engine)
customer_order_table.to_sql('cust',con=engine)


In [47]:
engine.execute('select b.*, a.rn==1 from cust b left join (select customer_id, credit_card_id, row_number() over (partition by customer_id order by c desc) as rn from (select customer_id, credit_card_id, count(order_id) c from cust group by customer_id, credit_card_id)) a on a.credit_card_id=b.credit_card_id ').fetchall()


[(0, 'C1027', 1, 37362, 10001, 1),
 (1, 'C1625', 2, 4857, 10026, 1),
 (2, 'C1027', 3, 37362, 10001, 1),
 (3, 'C1027', 4, 363, 100035, 0)]

In [21]:
product_table['Grocery_yesno'] = product_table.category.apply(lambda x: True if x in ['A','X','Y','P'] else False)


In [22]:
product_table

,product_id,category,Grocery_yesno
0,10001,A,True
1,10002,A,True
2,10003,B,False


In [31]:
grouped_customer = customer_order_table.groupby(['customer_id','credit_card_id']).agg({'order_id':'count'}).reset_index()
grouped_customer_2 = grouped_customer.sort_values(['customer_id','order_id'],ascending=False).groupby('customer_id').apply(lambda x: x.iloc[0,1:]).reset_index()
joined_customer_table = customer_order_table.merge(grouped_customer_2, on='customer_id')
# joined_customer_table['primaryCard_yesno'] = joined_customer_table['credit_card_id_x']==joined_customer_table['credit_card_id_y']


In [33]:
joined_customer_table

,customer_id,order_id_x,credit_card_id_x,product_id,credit_card_id_y,order_id_y
0,C1027,1,37362,10001,37362,2
1,C1027,3,37362,10001,37362,2
2,C1027,4,363,100035,37362,2
3,C1625,2,4857,10026,4857,1


In [24]:
joined_customer_table

,customer_id,order_id_x,credit_card_id_x,product_id,credit_card_id_y,order_id_y,primaryCard_yesno
0,C1027,1,37362,10001,37362,2,True
1,C1027,3,37362,10001,37362,2,True
2,C1027,4,363,100035,37362,2,False
3,C1625,2,4857,10026,4857,1,True


In [97]:
grouped_customer_2

,customer_id,credit_card_id,order_id
0,C1027,37362,2
1,C1625,4857,1


In [101]:
product_table.merge(joined_customer_table, on='product_id')

,product_id,category,Grocery_yesno,customer_id,order_id_x,credit_card_id_x,credit_card_id_y,order_id_y,primaryCard_yesno
0,10001,A,True,C1027,1,37362,37362,2,True
1,10001,A,True,C1027,3,37362,37362,2,True


In [51]:
m = [1,3,2,4]

In [52]:
m.sort()

In [53]:
m

[1, 2, 3, 4]

In [54]:
m.pop(3)

4

In [55]:
m

[1, 2, 3]

In [56]:
m.append(5)

In [57]:
m

[1, 2, 3, 5]

In [61]:
m.index(5)

3

In [59]:
m

[1, 2, 3, 5]

In [63]:
m.insert(2,4)

In [65]:
m.remove(3)

In [67]:
m.remove(2)

In [68]:
m

[1, 4, 5]

In [69]:
m.append(6)

In [88]:
m.reverse()

In [90]:
m.reverse()

In [75]:
m.insert(2,3)

In [79]:
d = {'m':2,'n':3}

In [86]:
d.update?